In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import re

In [0]:
df = spark.read.csv('dbfs:/FileStore/file-valid/sales_so_042022.csv', inferSchema=True, header=True)

In [0]:
display(df)

invoice_date,id,name,father,city,address,marks,phone,_c8
2022-04-04,1,sam,king,scranton,"11/497, Box st, Scranton, PA",67,-,null
2022-04-30,2,daniel,oliver,newyork,*#,34,76548263,null
2022-04-01,3,alex,joah,washington,-,55,65739302,null
2022-04-09,4,simon,nim,tokyo,"#456, North street, Tokyo",77,1417469493,2
2022-04-15,5,ross,james,paris,"Faux Ville, 1112, Paris",88,860485,fr
2022-04-16,6,joe,stephen,chicago,"Aswert, NY",51,675656573,null
2022-05-08,7,stephen,stephen,stephen,US,69,837-75846-8,null


In [0]:
# No of column validation
df.columns
if not len(df.columns) == 8:
    print('number of columns should be only 8')
else:
    print('number of columns equals to 8')

number of columns should be only 8


In [0]:
# data should be only in 7 cloumns
if False == df.select(*df.columns[8:]).isEmpty():
    print('data should be only in 8 columns')

data should be only in 8 columns


In [0]:
# Column order validation
tup1 = ('invoice_date', 'id', 'name', 'father', 'city', 'address', 'marks', 'phone')
tup2 = tuple(df.columns[:-1])

if not tup1==tup2: 
    print('column sequence is not in order')
else:
    print('column sequence is in order')

column sequence is in order


In [0]:
# Get date from file name
file_name = dbutils.fs.ls('dbfs:/FileStore/file-valid/sales_so_042022.csv')[0].name
print(file_name)
date = re.search(r"^([a-zA-Z\_]+)([\d]+)(\.csv)$",file_name).group(2)
# date1 = '202204'
# date2 = '042022'
date = re.search(r"([\d]{2})(20[\d]{2})|(20[\d]{2})([\d]{2})", date)
if date.group(1) == None:
    date = str(int(date.group(4)))
else:
    date = str(int(date.group(1)))

sales_so_042022.csv


In [0]:
# month in invoice date column should be same in filename

df_month = df.select('invoice_date', when(month('invoice_date')==date, True).otherwise(False).alias('month'))
if False == df_month.filter(df_month.month.contains(False)).collect()[0]['month']:
    print(f'month is not same as in file name')


month is not same as in file name


In [0]:
# invoice date should be in yyyy-mm-dd format

df_form = df.select('invoice_date', when(df.invoice_date == to_date('invoice_date', 'yyyy-mm-dd'), True).otherwise(False).alias('format_validate'))

try:
    if False == df_form.filter(df_form.format_validate.contains(False)).collect()[0]['format_validate']:
        print(f'invoice date should be in yyyy-mm-dd format')
except IndexError :
    print('date is in correct format')

date is in correct format


In [0]:
# marks should be in integer format, no alphabets or special characters allowed

df_marks = df.select('marks', when(df.marks == df.marks.astype(IntegerType()), True).otherwise(False).alias('marks_validate'))
try:
    if False == df_marks.filter(df_marks.marks_validate.contains(False)).collect()[0]['marks_validate']:
        print(f'marks should be in integer format, no alphabets or special characters allowed')
except IndexError :
    print('marks is in correct format')

marks is in correct format


In [0]:
df = df.withColumn('signal',when((col('name')==col('father'))|(col('name')==col('city'))|(col('father')==col('city')), False)
              .otherwise(True))
if False == df.filter(df.signal.contains(False)).collect()[0]['signal']:
    print(f'Unique Row Values Validation: name, father and city contain same values')

Unique Row Values Validation: name, father and city contain same values


## OR

In [0]:
count = df.withColumn('signal',when((col('name')==col('father'))|(col('name')==col('city'))|(col('father')==col('city')), False)
              .otherwise(True)).select('signal').distinct().count()
if count > 1:
    print(f'Unique Row Values Validation: name, father and city contain same values')

Unique Row Values Validation: name, father and city contain same values
